<a href="https://colab.research.google.com/github/VivekanandaMudelli/hand_drawn_sketch_recognition/blob/main/K_meanscluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gdown
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
file_id = "1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE"

# Construct the direct download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output = "data.csv"  # Change filename as needed
gdown.download(download_url, output, quiet=False)

# Read the CSV file
data = pd.read_csv(output)
data = data.dropna()

X = data.drop(columns = ['label'])
y = data['label']

print(X.shape)
print(y.shape)


Downloading...
From (original): https://drive.google.com/uc?id=1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE
From (redirected): https://drive.google.com/uc?id=1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE&confirm=t&uuid=7549efea-3d84-4f41-acd3-a6a19d08084e
To: /content/data.csv
100%|██████████| 943M/943M [00:21<00:00, 43.8MB/s]


(20180, 8100)
(20180,)


In [ ]:
# Mean
X_mean = X.mean()

# Standard deviation
X_std = X.std()

# Standardization
Z = (X - X_mean) / X_std.replace(0,1e-10)

In [ ]:
from sklearn.decomposition import PCA

# Taking top 300 dimensions
pca = PCA(n_components=50)
pca.fit(Z)
x_pca = pca.transform(Z)

# Create the dataframe
df = pd.DataFrame(x_pca,
                       columns=['PC{}'.
                       format(i+1)
                        for i in range(50)])
print(df)

             PC1        PC2        PC3        PC4        PC5        PC6  \
0     -13.473498  14.170485  -8.592970  -0.276518  -4.310717 -12.837164   
1      -7.392499  21.470417 -16.521421  -9.054983  10.826058   2.505740   
2     -11.364221   4.819343  -5.701317  -9.767067  -3.730633 -13.074421   
3      -9.464440   4.396213 -14.905355  -3.827589  -1.930125  -7.333181   
4      -3.514694   4.805839 -10.267748   6.538962  -6.552483   2.683379   
...          ...        ...        ...        ...        ...        ...   
20175  -3.464935   7.745460 -29.708590 -10.414632  14.356316   2.594418   
20176  -6.965887   8.512735  -5.489306   1.021767   0.134591  -6.490052   
20177 -10.654562  -4.086706  -9.890634  -5.559350   9.865370   8.734730   
20178   6.125788  -1.236722 -12.704954 -12.068882   0.344311  -0.449123   
20179  -3.484752  13.398968 -10.478203  -7.359174   4.313195  19.720054   

             PC7       PC8        PC9      PC10  ...       PC41      PC42  \
0       0.117520 -2.76

In [ ]:
# df['label'] = y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.40, random_state=42)# this will split the data into 87% train and 13% test data


In [ ]:
X_train = np.array(X_train, dtype=np.float64)  # Ensure all features are numeric


In [14]:
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm

# Ensure data is of correct type
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)







In [15]:
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)
def calculate_dist(x1, x2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt(np.sum((x1 - x2) ** 2))

def update_centroid(image_indexes, data_set):
    """Update centroid by averaging the points in a cluster."""
    if len(image_indexes) == 0:
        return np.zeros(data_set[0][0].shape, dtype=np.float64)  # Avoid division by zero

    centroid = np.zeros_like(data_set[0][0], dtype=np.float64)  # Adjust dimension dynamically
    for idx in image_indexes:
        centroid += data_set[idx][0]  # Sum all feature vectors
    centroid /= len(image_indexes)  # Get mean

    return centroid


In [16]:
# Number of clusters
K = 250
clusters = []

# Define K random points as initial cluster centroids
feature_dim = X_train.shape[1]  # Get the actual feature dimension
for i in range(K):
    point = np.random.uniform(-15, 15, size=(feature_dim,)).astype(np.float64)
    clusters.append([point, []])

# Prepare dataset as (features, label) tuples
data_set = [[feature.astype(np.float64), int(label)] for feature, label in zip(X_train, y_train)]

# K-means clustering (200 iterations)
for iteration in tqdm(range(200), desc="Clustering Iterations"):
    # Clear previous cluster assignments
    for cluster in clusters:
        cluster[1] = []

    # Assign each data point to the nearest centroid
    for i, (features, _) in enumerate(data_set):
        distances = [calculate_dist(cluster[0], features) for cluster in clusters]
        cluster_index = np.argmin(distances)  # Index of closest cluster
        clusters[cluster_index][1].append(i)  # Assign data point index

    # Update centroids based on current cluster assignments
    for i in range(K):
        if len(clusters[i][1]) > 0:  # Only update if cluster is not empty
            clusters[i][0] = update_centroid(clusters[i][1], data_set)

# Assign labels to clusters
cluster_labels = []
for i in range(K):
    labels_in_cluster = [data_set[idx][1] for idx in clusters[i][1]]
    if labels_in_cluster:  # Ensure cluster is not empty
        mode_label = stats.mode(labels_in_cluster, keepdims=True).mode[0]  # Extract mode correctly
    else:
        mode_label = -1  # Default label if cluster is empty
    cluster_labels.append(mode_label)

Clustering Iterations: 100%|██████████| 200/200 [1:17:34<00:00, 23.27s/it]


In [20]:
# Testing phase
from sklearn.metrics import accuracy_score

predictions = [] # Create a list to store predictions
for index in tqdm(range(X_test.shape[0]), total=len(y_test), desc="Testing Phase"): # Iterate over rows
    features = X_test.iloc[index].astype(np.float64)  # Ensure correct type by using iloc to get row data
    distances = [calculate_dist(cluster[0], features) for cluster in clusters]
    cluster_index = np.argmin(distances)  # Find nearest cluster
    predicted_label = cluster_labels[cluster_index]  # Assign label based on cluster
    predictions.append(predicted_label)

# Compute accuracy
accuracy = accuracy_score(y_test, predictions) # Pass predictions to accuracy_score
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Testing Phase: 100%|██████████| 8072/8072 [06:35<00:00, 20.39it/s]

Validation Accuracy: 11.74%


In [12]:
print(correct_predictions)

0


DBSCAN
